# GPU Optimization with Kernel Tuner

We start by downloading the data that we will use in this notebook.

In [ ]:
!wget -O GEMM_A100_cache.json.bz2 "https://github.com/KernelTuner/kernel_tuner_tutorial/blob/master/energy/data/GEMM_NVML_NVIDIA_A100-PCIE-40GB_freq_cache_fake_timings.json.bz2?raw=true"
!bunzip2 GEMM_A100_cache.json.bz2

Next, we import the required packages and set our defaults for plotting with Seaborn and Matplotlib:

In [ ]:
import kernel_tuner as kt
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = [15, 8]
sns.set_theme(style="whitegrid", palette="muted")
sns.set_context("paper", rc={"font.size":10,"axes.titlesize":9,"axes.labelsize":12})
sns.set(font_scale = 1.6)

### Tuning Setup

In this notebook, we will simulate the auto-tuning process of a highly-tunable matrix multiplication kernel. We start the tuning setup by defining the metrics we will use to measure performance.

In [ ]:
# Compute the number of operations that the matrix multiply performs
def ops(m, n, k):
    return (m * n * k * 2 + 2 * m * k) / 1e9

# Size of the matrices
m = n = k = 4096
problem_size = (512, 512)
total_flops = ops(m, n, k)

metrics = dict()
# Throughput
metrics["GFLOP/s"] = lambda p: total_flops / (p["time"] / 1000.0)
# Energy efficiency
metrics["GFLOPS/W"] = lambda p: total_flops / p["nvml_energy"]

Next, we define the parameters we would like to tune, their possible values, and restrictions that apply, creating the search space of possible configurations.

This includes the GPU core clock frequencies that we would like to test for each kernel configuration in the search space.
These are specified to Kernel Tuner with the special parameter name "nvml_gr_clock". Setting the GPU clock frequencies requires root privileges however. *That is why we will be simulating the tuning process for this hands-on.*

Kernel Tuner can simulate the optimization process for search spaces that have already been fully explored. The performance data collected by Kernel Tuner is stored in the data file that we have downloaded by running the first cell in this notebook.

In [ ]:
# Tunable parameters
tune_params = dict()
# The nvml_gr_clock is the tunable parameter affecting the GPU frequency in MHz, 690 is closest to the baseclock of 765
tune_params["nvml_gr_clock"] = [330, 510, 690, 870, 1050, 1230, 1410]

# Parameters needed for the code
tune_params["MWG"] = [16, 32, 64, 128]
tune_params["NWG"] = [16, 32, 64, 128]
tune_params["KWG"] = [32]
tune_params["MDIMC"] = [8, 16, 32]
tune_params["NDIMC"] = [8, 16, 32]
tune_params["MDIMA"] = [8, 16, 32]
tune_params["NDIMB"] = [8, 16, 32]
tune_params["KWI"] = [2]
tune_params["VWM"] = [1, 2, 4, 8]
tune_params["VWN"] = [1, 2, 4, 8]
tune_params["STRM"] = [0]
tune_params["STRN"] = [0]
tune_params["SA"] = [0, 1]
tune_params["SB"] = [0, 1]
tune_params["PRECISION"] = [32]

# Grid size
grid_div_x = ["MWG"]
grid_div_y = ["NWG"]
block_size_names = ["MDIMC", "NDIMC"]

# Search space restriction
restrict = []
restrict += ["KWG % KWI == 0"]
restrict += ["MWG % (MDIMC * VWM) == 0"]
restrict += ["NWG % (NDIMC * VWN) == 0"]
restrict += ["MWG % (MDIMA * VWM) == 0"]
restrict += ["NWG % (NDIMB * VWN) == 0"]
restrict += ["KWG % ((MDIMC * NDIMC)/MDIMA) == 0"]
restrict += ["KWG % ((MDIMC * NDIMC)/NDIMB) == 0"]
restrict += ["not (MWG == 128 and NWG == 128 and MDIMC == 8 and NDIMC == 8)"]

Next, we define a function that wraps the call to tune_kernel, because we will be calling it multiple times in this notebook. The function returns the best configuration, according to the current optimization objective, found during the tuning run.

In [ ]:
def get_optimal_config(objective: str, tune_parameters: dict, higher_is_better=True, strategy='genetic_algorithm', fevals=200) -> tuple[dict, list]:
    res_opt, env_opt = kt.tune_kernel(
        "Xgemm",
        "",
        problem_size,
        [],
        tune_parameters,
        block_size_names=block_size_names.copy(),
        simulation_mode=True,
        restrictions=restrict,
        grid_div_x=grid_div_x,
        grid_div_y=grid_div_y,
        strategy=strategy,
        strategy_options=dict(max_fevals=fevals),
        metrics=metrics,
        objective=objective,
        objective_higher_is_better=higher_is_better,
        cache="GEMM_A100_cache.json",
        quiet=True
    )
    return env_opt['best_config'], res_opt

Now that we have our function to easily call tune_kernel, let's have a look at tuning our matrix multiply kernel for different optimization objectives.


### Tuning for Time

We start by simply optimizing for the lowest possible kernel execution time.

In [ ]:
config_race_to_idle, res_race_to_idle = get_optimal_config("time", tune_params, higher_is_better=False)
config_race_to_idle['name'] = "race-to-idle (global)"

### Tuning for Time and Energy

The next step is to use our previous time-optimal configuration, and re-tune only the clock frequencies for energy efficiency. The idea behind this step is that we would like to use the best performing kernel configuration, but run it at the most energy-friendly clock frequency.

In [ ]:
# create a new dictionary of tunable parameters with only the clock frequencies
tune_params_only_clocks = tune_params.copy()
for key, value in config_race_to_idle.items():
    if key != "nvml_gr_clock" and key in tune_params_only_clocks:
        tune_params_only_clocks[key] = [value]

# tune the clock frequencies for energy efficiency
config_race_to_idle_plus_clocks, res_race_to_idle_plus_clocks = get_optimal_config('GFLOPS/W', tune_params_only_clocks, strategy='brute_force')
config_race_to_idle_plus_clocks['name'] = "race-to-idle + clocks"

### Tuning for Energy

The final step is to tune for energy efficiency. This optimization step attempts to find the configuration with the highest energy efficiency in the whole search space.

In [ ]:
config_energy_to_solution, res_energy_to_solution = get_optimal_config("GFLOPS/W", tune_params)
config_energy_to_solution['name'] = "energy-to-solution (global)"

### Plotting the Results

We can now look at the results in terms of energy efficiency per configuration in the bar chart below:

In [ ]:
df = pd.DataFrame([config_race_to_idle, config_race_to_idle_plus_clocks, config_energy_to_solution])
sns.barplot(x=df.nvml_energy, y=df.name, orient='h', hue=df.name, legend=False)
plt.xlabel('Energy (J), lower is better')
plt.ylabel('')
plt.title('Lowest energy configuration')
plt.tight_layout()

Finally, we can also make a scatterplot to show the relation between energy and time of our three different optimization efforts.

In addition, we can see the difference in how Kernel Tuner optimizes for performance and energy efficiency, and how the optimization algorithm improves over time, by plotting the configurations that have been explored during the optimization process.

In [ ]:
# plot our three configurations
sns.scatterplot(x=df['GFLOPS/W'], y=df['GFLOP/s'], hue=df.name, s=250)

# plot the configurations tried when optimizing for time
df_time = pd.DataFrame(res_race_to_idle)
sns.scatterplot(x=df_time['GFLOPS/W'], y=df_time['GFLOP/s'], hue=df_time.index, alpha=0.6, palette=sns.light_palette("midnightblue", as_cmap=True), legend=False)

df_time_clocks = pd.DataFrame(res_race_to_idle_plus_clocks)
sns.scatterplot(x=df_time_clocks['GFLOPS/W'], y=df_time_clocks['GFLOP/s'], alpha=0.9, color="orange", legend=False)

# plot the configurations tried when optimizing for energy
df_energy = pd.DataFrame(res_energy_to_solution)
sns.scatterplot(x=df_energy['GFLOPS/W'], y=df_energy['GFLOP/s'], hue=df_energy.index, alpha=0.6, palette=sns.light_palette("forestgreen", as_cmap=True), legend=False)

# finishing touches to the plot
plt.xlabel('GFLOPs per Watt, higher is better')
plt.ylabel('GFLOPs per second, higher is better')
plt.title('Energy versus Time')
plt.legend(title='', loc="upper left")
plt.tight_layout()

**Optional exercise:** Feel free to experiment, for example by running `get_optimal_config` again with more function evaluations or with different optimization strategies by passing different strategies in the `strategy=` option.

You can also try to define your own metrics and objectives. Why not optimize for time and energy in one go by creating an energy-delay-product metric?

**That's it! You've successfully completed the last hands-on of this workshop!**